# 1.0 Libraries & Dependencies 

In [ ]:
#installing dependencies for deepnote
!pip install xlrd
!pip install openpyxl
!pip install pandas plotly
!pip install wget 


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from PIL import Image
from io import BytesIO
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from IPython.display import display
from fuzzywuzzy import fuzz, process
from tabulate import tabulate  # Import the tabulate library
import plotly.graph_objects as go
import wget
from IPython import get_ipython

/root/venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Download the NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Arham's Code block



# 2.0 Notes For Employee Section Of ChatBot

### 2.1 Data Collections:

1. Item list - anonymized nike dataset used from previous job, redrafted with focus on Canadian market
2. Store List - Canadian Outlets (2019) using sources from citation list
2. Transactional Data - anonymized nike dataset used from previous job, redrafted with focus on Canadian market
3. Stock, Profits, leading time, backorder and dumpstock industry averages

### 2.2 Assumptions:

1. 16% profit margins [2], i.e. selling price without sale - cost of making the product 

### 2.3 Sources Referred:

1. https://d3.harvard.edu/platform-digit/submission/zara-achieving-the-fast-in-fast-fashion-through-analytics/
2. https://link.springer.com/chapter/10.1007/978-3-319-10912-1_221#:~:text=For%20example%2C%20profit%20margins%20for,higher%20for%20fast%20fashion%20retailers.

### 2.4 Features:

1. Top Dumpstock products to be consolidated & recommended strategies, Bottom Dumpstock & savings levers
2. Top products & Bottom products & categories by KPIs
3. Breakdown by stores, state, city to isolate business phenomenon

### 2.5 Employee Functions - Added by Arham:


Importing Datasets

In [ ]:
# importing datasets from public link on github
dataset_url = "https://github.com/arhamkun/ZaraChatbot/raw/a2130a78683b1da29e1815bb2f0ea8b4b80c0168/SupportingExcel1.xlsx"

#dataset1 transactional data
transactions = pd.read_excel(dataset_url, sheet_name='TransactionalData')

#dataset2 sku level data
products = pd.read_excel(dataset_url, sheet_name='productIDs')


Important Function Definitions

In [ ]:
## Employee - section A

# Function to display images and details
def display_product_info2(row, ax):
    response = requests.get(row['Zaraimg'])
    img = Image.open(BytesIO(response.content))

    ax.imshow(img)
    ax.set_title(f"{row['productName']}\nPrice: ${row['SellingPrice']}\nSold: {row['Sold']}/{row['Production']}",fontsize = 20)
    ax.axis('off')

# Top Products by Selling Price
def topProdBySP(p):
    products = p
    top_5_products = products.nlargest(5, 'SellingPrice')
    fig, axes = plt.subplots(1, len(top_5_products), figsize=(20, 8))  # Creating subplots

    for ax, (x, row) in zip(axes, top_5_products.iterrows()):
        display_product_info2(row, ax)

    plt.tight_layout()
    plt.show()
    
# Bottom Products by Selling Price
def bottomProdBySP(p):
    products = p
    top_5_products = products.nlargest(5, 'SellingPrice')
    fig, axes = plt.subplots(1, len(top_5_products), figsize=(20, 8))  # Creating subplots

    for ax, (x, row) in zip(axes, top_5_products.iterrows()):
        display_product_info2(row, ax)

    plt.tight_layout()
    plt.show()

    #
def topProdByDS(p):
    products = p
    top_5_products = products.nlargest(5, 'DeadStock Ratio')
    fig, axes = plt.subplots(1, len(top_5_products), figsize=(20, 8))  # Creating subplots

    for ax, (x, row) in zip(axes, top_5_products.iterrows()):
        display_product_info2(row, ax)

    plt.tight_layout()
    plt.show()
    
def bottomProdByDS(p):
    products = p
    top_5_products = products.nsmallest(5, 'DeadStock Ratio')
    fig, axes = plt.subplots(1, len(top_5_products), figsize=(20, 8))  # Creating subplots

    for ax, (x, row) in zip(axes, top_5_products.iterrows()):
        display_product_info2(row, ax)

    plt.tight_layout()
    plt.show()

    
## Employee - Section B

def citybreakdown(transactions_data):
    # Create a pandas DataFrame from the data
    transactions = pd.DataFrame(transactions_data)

    # Group data by city and calculate the total profit for each city
    city_summary = transactions.groupby('City')['Profits'].sum().reset_index()

    # Sort cities in descending order of contribution
    city_summary = city_summary.sort_values(by='Profits', ascending=False)

    # Calculate the total profit
    total_profit = city_summary['Profits'].sum()

    # Create a waterfall chart using Plotly
    fig = go.Figure()

    # Adding bars for each city's profit
    fig.add_trace(go.Waterfall(
        name='Contribution',
        x=city_summary['City'],
        textposition="outside",
        texttemplate='%{y:.0f}',  # Format text with 2 decimal points
        y=city_summary['Profits'],
        connector={"line": {"color": "rgb(63, 63, 63)"}},
    ))

    # Adding a total bar disconnected from other bars
    fig.add_trace(go.Waterfall(
        name='Total',
        x=["Total"],
        textposition="outside",
        texttemplate='%{y:.0f}',  # Format text with 2 decimal points
        y=[total_profit],
        base=0,  # Starting from 0 on y-axis
        connector={"line": {"color": "rgb(63, 63, 63)"}},
    ))

    # Update the layout to make it more presentable
    fig.update_layout(
        title="Waterfall Chart: City Profits Contribution",
        xaxis_title="City",
        yaxis_title="Profits",
        showlegend=True,
    )

    # Show the waterfall chart
    fig.show()


## Employee - Section C


#store Analysis

def storeAnalysis(transactions):
    
    # Group transactions by StoreID and aggregate sales, profits, and other metrics
    store_stats = transactions.groupby(['StoreName','StoreID']).agg(
        TotalSales=('Product_Price', 'sum'),
        TotalProfits=('Profits', 'sum'),
        UnitsSold=('TransactionID', 'count')
    ).reset_index()

    # Sort the store_stats DataFrame by TotalSales in descending order
    store_stats = store_stats.sort_values(by='TotalSales', ascending=False)

    # Find the store with the highest sale
    highest_sale_store = store_stats.loc[store_stats['UnitsSold'].idxmax()]

    # Find the store with the lowest sale
    lowest_sale_store = store_stats.loc[store_stats['TotalSales'].idxmin()]

    # Print the results
    print(f"Total Number of stores: {len(store_stats)}\n\n")
    
    print(f"\nStore with highest sale is store number {highest_sale_store['StoreID']} i.e, {highest_sale_store['StoreName']}"
          f"""\nRecommendation: 
          1. Maximize Top Sellers: Ensure high-demand products are well-stocked.
          2. Streamline Checkout: Expedite checkout process for seamless shopping.
          3. Community Events: Host events to engage and strengthen customer relationships.\n""")
    
    print(f"\nStore with lowest sale is store number {lowest_sale_store['StoreID']} i.e., {lowest_sale_store['StoreName']}"
          f"""\nRecommendation: 
          1. Trim Excess Stock: Reduce overstocked items.
          2. Promote Slow Sellers: Offer discounts on stagnant products.
          3. Forecast Demand Accurately: Improve inventory prediction.\n""")
    
    print(tabulate(store_stats, headers='keys', tablefmt='pretty', numalign='right'))


    
## Employee Admin Access

# Code block defines a function to check if the user is employee and returns true and false based on whether 
# the user is employee or not

# empkey to check whether the user is an employee or not
empkey = {'EMP1':'PASSWORD',
          'EMP2':'PASSWORD',
          'EMP3':'PASSWORD',
          'EMP4':'PASSWORD',
          'EMP5':'PASSWORD',
          'EMP6':'PASSWORD',
          'EMP7':'PASSWORD',
          'EMP8':'PASSWORD',
          'EMP9':'PASSWORD',
          'EMP10':'PASSWORD'
         }


# fn to check if the user is employee
def check_ifemployee():
    empkey = {
        'EMP1': 'PASSWORD',
        'EMP2': 'PASSWORD',
        'EMP3': 'PASSWORD',
        'EMP4': 'PASSWORD',
        'EMP5': 'PASSWORD',
        'EMP6': 'PASSWORD',
        'EMP7': 'PASSWORD',
        'EMP8': 'PASSWORD',
        'EMP9': 'PASSWORD',
        'EMP10': 'PASSWORD'
    }

    employee_code = input("Enter your employee code: ")
    password = input("Enter your password: ")

    if employee_code in empkey and password == empkey[employee_code]:
        print("Authentication successful. You are logged in as", employee_code)
        # Continue with further actions for authenticated employee
        return True
    else:
        print("Authentication failed. Access denied.")
        return False

# to Call the function
# check_ifemployee()



In [ ]:
##Arham's Main function

# functions for employee input categorization

# function A for deadstock
def function_A():
    print("Welcome! I'm here to provide information about deadstock.\n")
    
    while True:
        print("What would you like to know?")
        print("1. Top products with highest deadstock")
        print("2. Products with lowest deadstock")
        print("3. Products with highest sales")
        print("4. Products with lowest sales")
        print("0. Exit")
        
        user_input = input("Please enter your choice: ")
        
        products = pd.read_excel(dataset_url, sheet_name='productIDs')
        if user_input == '1':
            topProdByDS(products)
        elif user_input == '2':
            bottomProdByDS(products)
        elif user_input == '3':
            topProdBySP(products)
        elif user_input == '4':
            bottomProdBySP(products)
        elif user_input == '0':
            print("Exiting the program. Goodbye!")
            break
        else:
            print("\nInput not recognized, please try again.\n")
            continue
        
        while True:
            tempx = input("\nDo you want to continue (yes/no)? ").lower()
            if tempx == 'yes':
                break
            elif tempx == 'no':
                print("Exiting the program. Goodbye!")
                return
            else:
                print("Please enter 'yes' to continue or 'no' to exit.")

            
                
#function B for 
def function_B():
    citybreakdown(transactions)


# function c for store analysis
def function_C():
    storeAnalysis(transactions)

def categorize_input():
    print("Select a category:")
    print("1. Deadstock Analysis")
    print("2. City Breakdown")
    print("3. Store Analysis")
    print("4.View Influencer Insights") # keep this for arnav
    print("0. Exit")

    user_choice = input("Please enter your choice: ")

    if user_choice == '1':
        function_A()
    elif user_choice == '2':
        function_B()
    elif user_choice == '3':
        function_C()
    elif user_choice == '4':
        colab_insights()
    elif user_choice == '0':
        print("Exiting. Goodbye!")
    else:
        print("\nInvalid choice. Please select a valid option.\n")

dataset_url = "https://github.com/arhamkun/ZaraChatbot/raw/a2130a78683b1da29e1815bb2f0ea8b4b80c0168/SupportingExcel1.xlsx"

def employee_menu():
    if check_ifemployee() == True:
        while True:
            categorize_input()
            transactions = pd.read_excel(dataset_url, sheet_name='TransactionalData')
            products = pd.read_excel(dataset_url, sheet_name='productIDs')



# Arnav's Functions Below:

In [ ]:
def get_season(country):
    # Search for season information on Google
    search_query = f"current weather in {country}"
    search_url = f"https://www.google.com/search?q={search_query}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        result = soup.find("div", class_="BNeawe iBp4i AP7Wnd")
        
        if result:
            temperature = (result.get_text().lower())
            temperature = int(temperature.split('°')[0])
            if temperature < 10:
                season = "winter"
            elif 10 <= temperature < 15:
                season = "spring"
            elif 15 <= temperature < 20:
                season = "fall"
            else:
                season = "summer"
            return season
        else:
            print("Could not retrieve season information from Google.")
            return None
    else:
        print("Error: Unable to connect to Google.")
        return None

In [ ]:
def influencer_collaboration():
    print("Influencer Collaboration:")
    
    # Read data from "insta_data.csv" using pandas
    try:
        influencer_data = pd.read_csv("insta_data.csv")
    except FileNotFoundError:
        print("Error: Could not find 'insta_data.csv'. Make sure the file is in the same directory.")

    print("Influencer data loaded successfully.")

    instagram_name = input("Enter your Instagram name: ")
    user_influence = influencer_data[influencer_data["channel_info"] == instagram_name]

    score = int(user_influence["influence_score"])
    
    if user_influence.empty:
        print(f"Sorry, {instagram_name} not found in the influencer data.")
        return
    
    influence_score = int(user_influence["influence_score"].values[0])
    country=user_influence["country"].values[0]
      
    
    if influence_score>70:
    
        season = get_season(country)
        if season:
            collaboration_options = {
                "summer": "Share your creation on Instagram with the hashtag #SummerVibesChallenge and tag @ZaraFashionBot. Don't forget to mention why your outfit represents the perfect summer vibe.",
                "winter": "Show off your winter style! Use the hashtag #WinterFashionGlow and mention @ZaraFashionBot to join our winter fashion campaign.",
                "spring": "Spring into fashion! Tag @ZaraFashionBot in your spring-themed outfit post with the hashtag #SpringFreshLooks.",
                "fall": "Fall in love with fashion! Share your fall outfit using #FallFashionFaves and tag @ZaraFashionBot to be featured."
            }
            
            if season in collaboration_options:
                print(f"Congrats I see you qulaify for our collaboration program. I see the season in {country} is currently {season}")
                
                print("Collaboration Suggestion:")
                print(collaboration_options[season])
            else:
                print("No collaboration suggestion available for this season.")

In [ ]:
def colab_insights():
    while True:
        print("Welcome, what would you like to view?")
        print("1. View Top 20 Influencer by Brand Sentiment")
        print("2. View Correlation HeatMap")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3: ")

        if choice == "1":
            
            try:
                influencer_data = pd.read_csv("insta_data.csv")
            except FileNotFoundError:
                print("Error: Could not find 'insta_data.csv'. Make sure the file is in the same directory.")
                return
            try:
                sentiment_data = influencer_data[["channel_info", "Sentiment score"]]
                sorted_data = sentiment_data.sort_values(by="Sentiment score", ascending=False).head(20)
                
                names = sorted_data["channel_info"].values[::-1]  # Reverse the order of names
                scores = sorted_data["Sentiment score"].values[::-1]  # Reverse the order of scores
                
                plt.figure(figsize=(10, 8))
                plt.barh(names, scores, color='blue')
                plt.xlabel("Sentiment Score")
                plt.ylabel("Influencer Names")
                plt.title("Top 20 Collaboration Sentiment Insights")
                plt.tight_layout()
                plt.show()
            except KeyError:
                print("Error: 'Sentiment score' column not found in influencer data.")
                
                
        elif choice=="2":
            influencer_data = pd.read_csv("insta_data.csv")
            columns_to_plot = ["influence_score", "posts", "followers", "avg_likes", "60_day_eng_rate", "new_post_avg_like", "total_likes", "Sentiment score"]

            # Create a correlation matrix
            correlation_matrix = influencer_data[columns_to_plot].corr()

            # Set up the heatmap using Seaborn
            plt.figure(figsize=(10, 8))
            sns.heatmap(influencer_data.corr(), cbar=True, linewidths=0.5)
            plt.title("Correlation Heatmap")
            plt.show()
            
        elif choice == "3":
            print("Exiting")
            break

### Hot Item Recommendation - Kaz
**get_user_preferences Function:**
- This function is responsible for gathering user preferences related to trendy item recommendations.
- It prompts the user for their preferred category, color, size, and price range for clothing items.
- Returns the user's input as individual preference variables.

**trendy_item_recommendation Function:**
- This function provides recommendations for trendy clothing items based on user preferences.
- It takes user preferences as input arguments: category, color, size, and price preference.
- Filters the dataframe (presumably containing clothing item data) based on user preferences.
- Sorts the filtered items by popularity and trendiness scores in descending order.
- Recommends the top-rated item to the user, considering popularity and trendiness scores.
- Displays the recommended item's name, brand, description, price, category, and URL.
- Handles cases where no suitable items are found.

**Example Usage:**
- The code uses the `get_user_preferences` function to collect user preferences for clothing items.
- It then employs the `trendy_item_recommendation` function to provide a recommendation based on the user's input preferences.
- If a suitable item is found, it displays the details of the recommended item along with its URL.
- If no matching items are found, it notifies the user.

These functions work together to create an interactive experience where users provide their preferences, and the program offers recommendations based on those preferences.

In [ ]:
def get_user_preferences():
    print("Let's get started! Please provide your preferences:")

    user_category_preference = input(
        "Which category of hot items are you looking for? "
    )
    user_color_preference = input("What color do you prefer for your outfit? ")
    user_size_preference = input("What size do you wear? ")
    user_price_preference = input(
        "What's your preferred price category? (Low, Medium, High): "
    )

    return (
        user_category_preference,
        user_color_preference,
        user_size_preference,
        user_price_preference,
    )


In [ ]:
import pandas as pd

def trendy_item_recommendation(user_category_preference, user_color_preference, user_size_preference, user_price_preference):
    # Read the clothing item data from the CSV file
    df = pd.read_csv('Zara_Kaz_Functions.csv')
    
    # Display a welcome message
    print("Hi there! Let's find you some trendy items that match your preferences.")
    
    # Filter items based on user preferences
    filtered_items = df[
        (df['category'] == user_category_preference) &
        (df['color'] == user_color_preference) &
        (df['size'] == user_size_preference) &
        (df['Price_Category'] <= user_price_preference) &
        (df['availability'] == 'InStock')
    ]

    # Check if there are matching items
    if len(filtered_items) > 0:
        # Sort the filtered items by popularity and trendiness scores
        sorted_items = filtered_items.sort_values(by=['popularity_score', 'trendiness_score'], ascending=[False, False])
        
        # Check if sorted_items is not empty
        if len(sorted_items) > 0:
            # Get the details of the recommended item
            recommended_item = sorted_items.iloc[0]
            
            # Display the recommendation details
            print(f"I recommend '{recommended_item['name']}' by {recommended_item['brand']}!")
            print(f"Description: {recommended_item['description']}")
            print(f"Price: ${recommended_item['price']}")
            print(f"Category: {recommended_item['category']}")
            print(f"You can check it out [here]({recommended_item['url']}).")
        else:
            print("I'm sorry, I couldn't find any trendy items matching your preferences.")
    else:
        print("I'm sorry, I couldn't find any items matching your preferences.")


### Event Based Recommendation - Kaz

**match_event Function:**
- Compares the input event with a predefined list of event categories.
- Uses fuzzy string matching to find the best-matched event category.
- Returns the matched event category and a matching score.

**filter_recommendations Function:**
- Filters recommended items based on user preferences for color, size, and price category.
- Takes recommended_items DataFrame and user preferences as inputs.
- Returns a DataFrame containing filtered items.

**find_complimentary_items Function:**
- Finds items that complement the recommended item.
- Takes clothing_data, recommended_item, matched_event, user_color_preference, and user_size_preference as inputs.
- Returns a DataFrame containing complementary items based on specified criteria.

**event_based_recommendation Function:**
- Guides the user through finding an outfit for a specific event.
- Takes an event and clothing_data as inputs.
- Uses fuzzy string matching to find a matching event category.
- Displays a welcome message and event category based on the match.
- Prompts the user for color, size, and price range preferences.
- Filters recommendations based on user preferences and event category.
- Provides recommendations and, if available, complementary item details.

These functions work together to provide a conversational and interactive experience for users, guiding them to find the perfect outfit for a specific event based on their preferences. The recommendations are displayed in a structured format with relevant information.

In [ ]:
def match_event(event):
    # List of event categories
    event_lists = [
        "Casual and Everyday Wear",
        "Formal and Business Events",
        "Evening and Party Wear",
        "Beach and Vacation",
        "Athletic and Workout",
        "Sleep and Loungewear",
        "Winter and Cold Weather",
        "Summer and Hot Weather",
        "Festive and Holiday Wear",
    ]
    # Find the best-matched event category
    matched_event, score = process.extractOne(
        event.lower(), event_lists, scorer=fuzz.partial_ratio
    )
    return matched_event, score


In [ ]:
def filter_recommendations(recommended_items, user_color_preference, user_size_preference, user_price_preference):
    # Filter recommended items based on user preferences
    filtered_items = recommended_items[
        (recommended_items['color'] == user_color_preference) &
        (recommended_items['size'] == user_size_preference) &
        (recommended_items['price_category'] == user_price_preference)
    ]
    return filtered_items

In [ ]:
def find_complimentary_items(clothing_data, recommended_item, matched_event, user_color_preference, user_size_preference):
    # Find items that complement the recommended item
    recommended_category = recommended_item['category'].item()
    complimentary_category = recommended_item['complimentary_clothing_category'].item()

    complimentary_items = clothing_data[
        (clothing_data['category'] == complimentary_category) &
        (clothing_data['event_category'] == matched_event) &
        (clothing_data['color'] == user_color_preference) &
        (clothing_data['size'] == user_size_preference) &
        (clothing_data['availability'] == 'InStock')
    ]
    return complimentary_items

In [ ]:
def event_based_recommendation(event, clothing_data):
    # Introduction message
    print("Hello there! Let's find the perfect outfit for your event.")
    
    # Match the input event to the closest event category
    matched_event, score = match_event(event)

    # Check if the match score is sufficient
    if score >= 70:
        # Display matched event category
        print(f"Great! It looks like you're attending a '{matched_event}' event.")
        
        # Filter recommended items for the matched event and availability
        recommended_items = clothing_data[clothing_data['event_category'] == matched_event]
        recommended_items = recommended_items[recommended_items['availability'] == 'InStock']

        # Gather available colors, sizes, and price ranges
        available_colors = recommended_items['color'].unique()
        available_sizes = recommended_items['size'].unique()
        available_price_range = recommended_items['price_category'].unique().tolist()

        # Collect user preferences
        user_color_preference = input(f"Available colors: {', '.join(available_colors)}. What's your preferred color? ").upper()
        user_size_preference = input(f"Available sizes: {', '.join(available_sizes)}. What's your preferred size? ").upper()
        print('Price Range: low: < $40, medium: < $80, high < $120, luxury +$120')
        user_price_preference = input(f"Available price range: {', '.join(available_price_range)}. What's your preferred price range? ").upper()

        # Filter recommended items based on user preferences
        recommended_items = filter_recommendations(recommended_items, user_color_preference, user_size_preference, user_price_preference)

        if len(recommended_items) > 0:
            # Get a recommended item
            recommended_item = recommended_items.sample(n=1)
            
            # Find a complementary item
            complimentary_item = find_complimentary_items(clothing_data, recommended_item, matched_event, user_color_preference, user_size_preference)

            if len(complimentary_item) >= 1:
                # Display recommendation and complementary item details
                print('Here is our recommendation:')
                print(f"Name: {recommended_item['name'].values[0]}")
                print(f"Size: {recommended_item['size'].values[0]}")
                print(f"Color: {recommended_item['color'].values[0]}")
                print(f"Price: {recommended_item['price'].values[0]}")
                print(f"URL: {recommended_item['url'].values[0]}")
                
                print('\nYou might also be interested in this:')
                print(f"Name: {complimentary_item['name'].values[0]}")
                print(f"Size: {complimentary_item['size'].values[0]}")
                print(f"Color: {complimentary_item['color'].values[0]}")
                print(f"Price: {complimentary_item['price'].values[0]}")
                print(f"URL: {complimentary_item['url'].values[0]}")
            else:
                # Display recommendation details when no complementary item is found
                print('Here is our recommendation:')
                print(f"Name: {recommended_item['name'].values[0]}")
                print(f"Size: {recommended_item['size'].values[0]}")
                print(f"Color: {recommended_item['color'].values[0]}")
                print(f"Price: {recommended_item['price'].values[0]}")
                print(f"URL: {recommended_item['url'].values[0]}")
                
        else: 
            # No suitable recommendations found
            print("I'm sorry, I couldn't find suitable recommendations for that event with your preferences.")
    else:
        # No suitable event match found
        print("I'm sorry, I couldn't find suitable recommendations for that event.")

# this is a scenario where complimentray works:   casual, blue, size L, price medium

In [ ]:
def customer_menu():
    while True:
        print("Welcome, Customer! What would you like help with today?") # kaz edit here
        print("1. Hot items")
        print("2. Event Based Reccommendations")
        print("3. Influencer Collaboration")
        print("4. Exit")
        choice = input("Enter your choice (1/2/3/4): ")
        
        if choice == "1":
            user_category_preference, user_color_preference, user_size_preference, user_price_preference = get_user_preferences()
            trendy_item_recommendation(user_category_preference, user_color_preference, user_size_preference, user_price_preference)            
        elif choice == "2":
            clothing_data = pd.read_csv('Zara_Kaz_Functions')
            event = input("Hello! What's the occasion you're dressing up for? ")
            event_based_recommendation(event, clothing_data)
        elif choice == "3":
            influencer_collaboration()
        elif choice == "4":
            print("Exiting customer menu.")
            break
        else:
            print("Invalid choice.")

In [ ]:
# def employee_menu():

# #Arhams employer id check


#     while True:
#         print("Welcome, Employee! What would you like to do?")
#         print("1.View Influencer Insights") # keep this for arnav
#         print("2. Exit")
#         choice = input("Enter your choice (1/2/3): ")

#         if choice == "1":
#             colab_insights() # keep this for arnav







#         elif choice == "2":
#             print("Exiting employee menu.")
#             break
#         else:
#             print("Invalid choice.") 

# Main Function

The code snippet defines the main() function, which serves as the entry point for the program. It sets up an infinite loop that continues until the user decides to exit. Within the loop, the chatbot displays a welcome message and prompts the user to choose their role:

Employee: If the user selects this option, the program calls the employee_menu() function. This function will contain a menu tailored to employee interactions.

Customer: If the user chooses this option, the program calls the customer_menu() function. This function will contain a menu tailored to customer interactions.

Exit: If the user decides to exit, the program displays a farewell message and terminates the loop.

In [ ]:
def main():
   
        
    while True:
        print("Hi! Welcome to the Zara chatbot. Before we begin, please choose if you are an employee or customer.")
        print("1. Employee")
        print("2. Customer")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ")

        if choice == "1":
            employee_menu()

        elif choice == "2":
            customer_menu() #function call

        elif choice == "3":
            print("Goodbye! Have a great day.")
            break
    
        else:
            print("Invalid choice. Please select '1' for Employee, '2' for Customer, or '3' to Exit.")

       
if __name__ == "__main__":
    main()

Hi! Welcome to the Zara chatbot. Before we begin, please choose if you are an employee or customer.
1. Employee
2. Customer
3. Exit


KeyboardInterrupt: Interrupted by user

In [ ]:
# Provide the path to the Zara logo image on your system
image_path = "https://i.pinimg.com/originals/5f/f8/aa/5ff8aa286413e2b47204efc3dae0a8f3.jpg"

# Open the image
img = Image.open(image_path)

# Resize the image to the cover dimensions
cover_dimensions = (800, 400)  # Adjust the dimensions as needed
img = img.resize(cover_dimensions)

# Display the image
img.show()

NameError: name 'Image' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=efa188a0-2f94-4804-9f19-625dc30c6533' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>